<a href="https://colab.research.google.com/github/VladOS95-cyber/deeplearning_practice/blob/main/AutoEncoders_RecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AutoEncoders

##Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [3]:
movies = pd.read_csv('movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

## Preparing the training set and the test set


In [4]:
training_set = pd.read_csv('u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

## Getting the number of users and movies


In [5]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

## Converting the data into an array with users in lines and movies in columns


In [6]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [7]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Creating the architecture of the Neural Network


In [20]:
class SAE(nn.Module):
  def __init__(self, ):
    super().__init__()
    self.fc1 = nn.Linear(nb_movies, 20)
    self.fc2 = nn.Linear(20, 10)
    self.fc3 = nn.Linear(10, 20)
    self.fc4 = nn.Linear(20, nb_movies)
    self.activation = nn.Sigmoid()

  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.fc4(x)
    return x

sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), weight_decay=0.5)

## Training the SAE


In [27]:
# loss should be around 0.91 - 0.92 which means accuracy is about 1 star
# in 5 stars recommendation system
# Example: if movie has 4 stars rating, prediction accuracy will be between 3 - 4 stars
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.requires_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.item() * mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss/s))

epoch: 1 loss: 1.0891801119652067
epoch: 2 loss: 1.051624996993494
epoch: 3 loss: 1.037828452825287
epoch: 4 loss: 1.0304022691091614
epoch: 5 loss: 1.0263437079106443
epoch: 6 loss: 1.0237325711187824
epoch: 7 loss: 1.0218833048539018
epoch: 8 loss: 1.0204232395913777
epoch: 9 loss: 1.0197886512098622
epoch: 10 loss: 1.0188054981041197
epoch: 11 loss: 1.018275011461171
epoch: 12 loss: 1.0178276519250935
epoch: 13 loss: 1.0177998931512446
epoch: 14 loss: 1.0169903705605114
epoch: 15 loss: 1.0169910244111313
epoch: 16 loss: 1.01686647916472
epoch: 17 loss: 1.0165867345153958
epoch: 18 loss: 1.0162666728914274
epoch: 19 loss: 1.0164285524622232
epoch: 20 loss: 1.0159558981423467
epoch: 21 loss: 1.0159363795938177
epoch: 22 loss: 1.0158750838319468
epoch: 23 loss: 1.0158760269761486
epoch: 24 loss: 1.0155640523813894
epoch: 25 loss: 1.0156731109353485
epoch: 26 loss: 1.0151764916766668
epoch: 27 loss: 1.0152892949087593
epoch: 28 loss: 1.0145761069873196
epoch: 29 loss: 1.012046387773822


## Testing the SAE


In [29]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.requires_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.item() * mean_corrector)
    s += 1.
print('Test loss: ' + str(test_loss/s))

Test loss: 0.9510461220127633
